In [1]:
using HorizonSideRobots

In [2]:
a = Robot("untitled.sit", animate=true);

┌ Warning: Found `resolution` in the theme when creating a `Scene`. The `resolution` keyword for `Scene`s and `Figure`s has been deprecated. Use `Figure(; size = ...` or `Scene(; size = ...)` instead, which better reflects that this is a unitless size and not a pixel resolution. The key could also come from `set_theme!` calls or related theming functions.
└ @ Makie C:\Users\Alex\.julia\packages\Makie\Qvk4f\src\scenes.jl:220


In [3]:
function try_move!(r, side)
    if !isborder(r, side)
        move!(r, side)
        return true
    end
    return false
end

try_move! (generic function with 1 method)

In [4]:
function inverse(side)
    if side == Sud
        return Nord
    end
    if side == Nord
        return Sud
    end
    if side == Ost
        return West
    end
    if side == West
        return Ost
    end
end

inverse (generic function with 1 method)

In [5]:
function HorizonSideRobots.move!(r, side, n_steps)
    stepped = 0
    for _ in range(1, n_steps)
        if try_move!(r, side)
            stepped += 1
        end
    end
    return stepped
end

In [6]:
function to_start!(r, side1, side2, steps_arr)
    side1_1 = inverse(side1)
    side2_1 = inverse(side2)
    side = side2
    for i in range(-length(steps_arr), -1)
        move!(r, side, steps_arr[abs(i)])
        if side == side2
            side = side1
        else
            if side == side1
                side = side2_1
            else
                if side == side2_1
                    side = side1_1
                else
                    if side == side1_1
                        side = side2
                    end
                end
            end
        end
    end
end

to_start! (generic function with 1 method)

In [7]:
function mark_line!(r, side1, side2)
    side1_cnt = 0
    side2_cnt = 0
    arr = []
    putmarker!(r)
    flag = 0
    while !(isborder(r, side1) && isborder(r, side2))
        moved_1 = try_move!(r, side1)
        moved_2 = try_move!(r, side2)
        if !(moved_1 || moved_2)
            break
        end
        if moved_1 && moved_2
            putmarker!(r)
            append!(arr, 1)
            append!(arr, 1)
            append!(arr, 0)
            append!(arr, 0)
        else
            if moved_1
                side1_cnt += 1
                while isborder(r, side2) && !isborder(r, side1)
                    if try_move!(r, side1)
                        side1_cnt += 1
                    end
                end
                if !(isborder(r, side1) && isborder(r, side2))
                    move!(r, side2)
                    if isborder(r, inverse(side1))
                        move!(r, side2, side1_cnt-1)
                        putmarker!(r)
                        append!(arr, side1_cnt)
                        append!(arr, side1_cnt)
                        append!(arr, 0)
                        append!(arr, 0)
                    else
                        move!(r, inverse(side1), side1_cnt-1)
                        putmarker!(r)
                        append!(arr, side1_cnt)
                        append!(arr, 1)
                        append!(arr, side1_cnt-1)
                        append!(arr, 0)
                    end
                else
                    append!(arr, side1_cnt)
                    append!(arr, 0)
                    append!(arr, 0)
                    append!(arr, 0)
                end
                side1_cnt = 0
            end
            if moved_2
                side2_cnt += 1
                while isborder(r, side1) && !isborder(r, side2)
                    if try_move!(r, side2)
                        side2_cnt += 1
                    end
                end
                if !(isborder(r, side1) && isborder(r, side2))
                    move!(r, side1)
                    if isborder(r, inverse(side2))
                        move!(r, side1, side2_cnt)
                        putmarker!(r)
                        append!(arr, 0)
                        append!(arr, side2_cnt)
                        append!(arr, 0)
                        append!(arr, 0)
                        
                        append!(arr, side2_cnt)
                        append!(arr, 0)
                        append!(arr, 0)
                        append!(arr, 0)
                    else
                        move!(r, inverse(side2), side2_cnt-1)
                        putmarker!(r)
                        append!(arr, 0)
                        append!(arr, side2_cnt)
                        append!(arr, 0)
                        append!(arr, 0)

                        append!(arr, 1)
                        append!(arr, 0)
                        append!(arr, 0)
                        append!(arr, side2_cnt-1)
                    end
                else
                    append!(arr, 0)
                    append!(arr, side2_cnt)
                    append!(arr, 0)
                    append!(arr, 0)
                end
                side2_cnt = 0
            end
        end
    end
    to_start!(r, side1, side2, arr)
end

mark_line! (generic function with 1 method)

In [8]:
function main!(r)
    mark_line!(r, Ost, Nord)
    mark_line!(r, Ost, Sud)
    mark_line!(r, West, Nord)
    mark_line!(r, West, Sud)
end

main! (generic function with 1 method)

In [9]:
main!(a)